In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!./Miniconda3-latest-Linux-x86_64.sh -b -p /usr/local -f

In [3]:
! conda install -c milagraph -c conda-forge graphvite python=3.7 cudatoolkit=10.1 -y 

mkl-2020.2           | 138.3 MB  | : 100% 1.0/1 [00:25<00:00, 25.53s/it]               
python-dateutil-2.8. | 220 KB    | : 100% 1.0/1 [00:00<00:00, 12.89it/s]
gflags-2.2.2         | 114 KB    | : 100% 1.0/1 [00:00<00:00, 19.90it/s]
click-7.1.2          | 64 KB     | : 100% 1.0/1 [00:00<00:00, 23.81it/s]
cycler-0.10.0        | 9 KB      | : 100% 1.0/1 [00:00<00:00, 24.91it/s]
pyparsing-2.4.7      | 60 KB     | : 100% 1.0/1 [00:00<00:00, 31.89it/s]
graphvite-0.2.2      | 52.6 MB   | : 100% 1.0/1 [00:11<00:00, 11.23s/it]
torchvision-0.4.2    | 5.8 MB    | : 100% 1.0/1 [00:00<00:00,  3.90it/s]
easydict-1.9         | 9 KB      | : 100% 1.0/1 [00:00<00:00, 28.66it/s]
numpy-base-1.19.2    | 4.1 MB    | : 100% 1.0/1 [00:00<00:00,  4.60it/s]
pip-21.0.1           | 1.1 MB    | : 100% 1.0/1 [00:00<00:00,  1.32it/s]
Preparing transaction: | / - \ | done
Verifying transaction: - \ | / - \ | / - \ | / done
Executing transaction: \ | / - \ | / - \ 

In [ ]:
#!conda install wurlitzer ipykernel -y
! pip install wurlitzer
! pip install ipykernel

In [5]:
import site
site.addsitedir("/usr/local/lib/python3.7/site-packages")
%reload_ext wurlitzer

import torch
from graphvite.application.network import LinkPredictor

In [6]:
cd drive/MyDrive/Graph_Link_Prediction/graphvite/

/content/drive/MyDrive/Graph_Link_Prediction/graphvite


In [7]:
! graphvite run my_config.yml

loading graph from data/feature.txt
0.0048671%
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
Graph<uint32>
------------------ Graph -------------------
#vertex: 3816, #edge: 16090
as undirected: yes, normalization: no
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
[time] GraphApplication.load: 0.514537 s
[time] GraphApplication.build: 0.392786 s
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
GraphSolver<32, float32, uint32>
----------------- Resource -----------------
#worker: 1, #sampler: 1, #partition: 1
tied weights: no, episode size: 200
gpu memory limit: 14.6 GiB
gpu memory cost: 42.2 MiB
----------------- Sampling -----------------
augmentation step: 5, shuffle base: 5
random walk length: 40
random walk batch size: 100
#negative: 1, negative sample exponent: 0.75
----------------- Training -----------------
model: LINE
optimizer: SGD
learning rate: 0.025, lr schedule: linear
weight decay: 0.005
#epoch: 4000, batch size: 100000
resume: no
positive reuse: 1, negative weight: 5
>>>>>>>>>

In [8]:
# Load the pre-trained model's path for future steps.
import pickle
from graphvite.application.network import LinkPredictor

emb = 'model/submit.pkl'
model = pickle.load(open(emb, 'rb'))
mapping = model.graph.name2id
model = LinkPredictor('LINE', model.solver.vertex_embeddings, model.solver.context_embeddings)

# Test the model on validation.


In [ ]:
import numpy
# Loading evaluating set.
Y = []
score = []
with open('data/test2.txt', 'r') as f:
    
    for line in f:
        h, t, y = line.split()
        if h not in mapping or t not in mapping:
            score.append(-5.0)
            Y.append(int(y))
            continue
        x1 = torch.tensor([mapping[h]])
        y1 = torch.tensor([mapping[t]])
        pred = model(x1, y1)
        pred = pred[0].tolist()
        score.append(pred)
        Y.append(int(y))

In [ ]:
import math


sigmoid_vec = []
for i in range(len(score)):
    sigmoid_vec.append(1/(1 + math.exp(-score[i])))

# Setting threshold and making predictions.
threshold = 0.1
prediction = []
for i in range(len(score)):
    output = 0
    if sigmoid_vec[i] > threshold:
        output = 1
    prediction.append(output)


print(Y[:3], prediction[:3])
print(len(Y), len(prediction))
print(type(Y[0]), type(prediction[0]))

[1, 1, 1] [1, 1, 1]
23368 23368
<class 'int'> <class 'int'>


In [ ]:
from sklearn.metrics import classification_report
report = classification_report(Y, prediction, zero_division=0, digits=4)

from sklearn.metrics import roc_curve, auc, roc_auc_score
false_positive_rate, true_positive_rate, thresholds = roc_curve(Y, prediction)
auc = auc(false_positive_rate, true_positive_rate)

print(auc)
report += ('auc = ' + str(auc))
print(report)

0.9309311879493325
              precision    recall  f1-score   support

           0     1.0000    0.8619    0.9258     11684
           1     0.8786    1.0000    0.9354     11684

    accuracy                         0.9309     23368
   macro avg     0.9393    0.9309    0.9306     23368
weighted avg     0.9393    0.9309    0.9306     23368
auc = 0.9309311879493325


# OutPut the prediction to csv for Kaggle.


In [9]:
H, T = [], []
Kaggle_pred = []
with open('data/test-public.csv', 'r') as f:
    next(f)
    for line in f:
        _, h, t = line.split(',')
        h = str(int(h))
        t = str(int(t))
        if h not in mapping or t not in mapping:
            Kaggle_pred.append(-5.0)
            continue
        x1 = torch.tensor([mapping[h]])
        y1 = torch.tensor([mapping[t]])
        pred = model(x1, y1)
        pred = pred[0].tolist()
        Kaggle_pred.append(pred)

print(Kaggle_pred)

[0.10157689452171326, -4.406339168548584, -3.6867477893829346, 0.8390579223632812, -4.29557466506958, -4.38645076751709, -4.717746257781982, 1.212251901626587, -4.557036876678467, 2.706514358520508, -1.8311090469360352, 2.3284153938293457, -5.6005353927612305, -3.145246982574463, 1.6961067914962769, -3.5906755924224854, -1.3602116107940674, -1.5510517358779907, 1.2155399322509766, 1.9901394844055176, 0.950931191444397, -3.7846295833587646, -1.2781485319137573, -4.725695610046387, -4.927781581878662, 2.0420327186584473, 2.158909320831299, 1.9820325374603271, -2.5385005474090576, -4.04141902923584, -3.765880584716797, -3.0918774604797363, -3.0750138759613037, -0.10260975360870361, -4.003097057342529, 1.0152955055236816, -3.492633819580078, -3.3360366821289062, -0.713817834854126, -3.6286001205444336, -3.8572921752929688, -2.145460605621338, -3.116055965423584, 2.082535982131958, -3.7678771018981934, 2.1167545318603516, -2.8956940174102783, 1.968846321105957, 1.7753822803497314, 1.8896260

In [10]:
# save the output
out = 'right_one.csv'
with open(out, 'w') as o:
    o.write('Id,Predicted\n')
    for i, s in enumerate(Kaggle_pred):
        o.write(f'{i+1},{s}\n')
print('Write output to', out)

Write output to right_one.csv


# 接下来采用Logistic Regression试一下



In [ ]:
import pickle
import os
import numpy as np

def get_model():
    # Loading trained embeddings
    emb = 'model/LINE_full.pkl'
    model = pickle.load(open(emb, 'rb'))
    return model

In [ ]:
# Generating input vectors
def get_datavec(model):
    
    mapping = model.graph.name2id
    context_emb = model.solver.context_embeddings
    vertex_emb = model.solver.vertex_embeddings
    
    # Reading file and generating input vectors
    file_name = r'data/edges.txt'
    X, Y = [], []
    with open(file_name, 'r') as f:
        for line in f:
            h, t, y = line.split()
            if h not in mapping:
                X_vec = list(np.zeros(32))
            else:
                X_vec = list(vertex_emb[mapping[h]])
            if t not in mapping:
                X_vec.extend(np.zeros(32))
            else:
                X_vec.extend(context_emb[mapping[t]])
            X.append(X_vec)
            Y.append(int(y))
    return X, Y

In [ ]:
from sklearn.model_selection import train_test_split
import pickle
import pandas as pd
from sklearn.metrics import classification_report

# Loading full data
model = get_model()
X, Y =  get_datavec(model)
# Spliting data into training set and testing set
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size = 0.2)

ValueError: ignored

In [ ]:
# Changing data to ndarray format
import numpy as np
Xtrain = np.array(Xtrain)
Xtest = np.array(Xtest)

# Setting LR model and training the model
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver = 'sag')
log_reg.fit(Xtrain, Ytrain)

# Making prediction on the testing set
prediction = log_reg.predict(Xtest)

# Evaluating the model
report = classification_report(Ytest, prediction, zero_division = 0, digits = 4)

from sklearn.metrics import roc_curve, auc, roc_auc_score
false_positive_rate, true_positive_rate, thresholds = roc_curve(Ytest, prediction)
auc = auc(false_positive_rate, true_positive_rate)

report += ('auc = ' + str(auc))
print(report)

NameError: ignored